In [5]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import urllib
import json
import os, glob
from IPython.core.display import display, HTML
import logging

import warpImage
from pointAnnotator import pointAnnotator, pointCopier
import helpers
try:
    import config as cfg
    print('loaded device specific config')
except ImportError as E:
    import default_config as cfg
    print('loaded default config')


display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib notebook

loaded default config


In [6]:
localWarp=False

#for some reason, I find you have to run this line twice to actually take effect (depending on matplotlib version)
%matplotlib notebook 

In [14]:
#get path to network folder with lims images for this mouse
mouse_num = '415149'
json_string = urllib.urlopen("http://lims2/specimens/isi_experiment_details/" + mouse_num + ".json").read()
info = json.loads(json_string)
print(info)

####################it looks like it always pulls the map with targets on it but what happens if two have targets?
vimg_path = r'\\' + info[0]['target_map_image_path'][1:]
#experiments = info[0]['isi_experiments']
#for exp in experiments:
#    if exp['targets']['insertion_targets']['reticle_space'] is not None:
#        pass

print(vimg_path)

[{u'name': u'C57BL/6J-415149', u'isi_image_overlay_path': u'/allen/programs/braintv/production/neuralcoding/prod56/specimen_753795610/isi_experiment_760659575/760659575_isi_overlay.tif', u'isi_experiments': [{u'project_code': u'NeuropixelVisualCoding', u'targets': {u'actual_insertion': {u'reticle_space': [{u'y': -1066.63, u'x': 904.29}, {u'y': -507.63, u'x': 1707.42}, {u'y': 733.46, u'x': 899.98}, {u'y': 224.82, u'x': -1649.91}, {u'y': -77.25, u'x': -1331.14}, {u'y': -701.03, u'x': -99.32}], u'image_space': [{u'y': 544.0, u'x': 1584.0}, {u'y': 788.0, u'x': 1944.0}, {u'y': 1344.0, u'x': 1592.0}, {u'y': 1132.0, u'x': 456.0}, {u'y': 996.0, u'x': 596.0}, {u'y': 712.0, u'x': 1140.0}]}, u'insertion_targets': {u'reticle_space': [{u'y': -1150.74, u'x': 653.31}, {u'y': -489.89, u'x': 960.14}, {u'y': 487.06, u'x': 263.98}, {u'y': 320.37, u'x': -1565.59}, {u'y': -221.02, u'x': -1311.36}, {u'y': -995.35, u'x': -244.18}], u'image_space': [{u'y': 508.0, u'x': 1472.0}, {u'y': 800.0, u'x': 1612.0}, {u

In [8]:
#get vasculature image
vimg = cv2.imread(vimg_path)

#get probe insertion image
pimgFileList = [r"Z:\03132019_416656\2019_03_13_15_11_10_left.png"]
if len(pimgFileList) == 0:
    pimgFileList = cfg.get_insertion_image_paths(mouse_num)
    
canidate_image_count = len(pimgFileList)

In [9]:


pimg_dict = {}
ppa_dict = {}
points_dict = {}


if canidate_image_count > 1:
    fig, ax = plt.subplots(canidate_image_count, figsize=(20,20/(2*aspect)))
    for idx, imgage_path in enumerate(pimgFileList):
        next_ax = ax[idx]
        next_image = cv2.imread(imgage_path)
        pimg_dict[imgage_path] = next_ax.imshow(next_image, origin='upper')
        ppa_dict[imgage_path] = pointAnnotator(pimg_dict[imgage_path], next_ax)
        image_name = os.path.split(image_path)[0]
        points_dict[imgage_path] = helpers.points_from_path(directory_path, image_name)
        ppa_dict[imgage_path].loadPoints(points_dict[imgage_path])
        


NameError: name 'pimgFileName' is not defined

In [ ]:
#Display the vasculature images and select points of interest (at least three)

if canidate_image_count > 1:
    count = 0
    for image_path, annotator in ppa_dict.items():
        if not(annotator.points == points_dict[image_path])
            count+=1
            chosen_image_path = image_path
    try:
        assert(count==1)
    except AssertionError as E:
        logging.warning('Image chosen is ambiguous, none or multiple were clicked. Please reset and try again.')
        raise(E)
    pimg_path = chosen_image_path
else:
    pimg_path = pimgFileList[0]

pimg = cv2.imread(pimg_path)
chosen_probe_image_name = os.path.split(pimg_path)[1]
chosen_vasculature_image_name = os.path.split(vimg_path)[1]

#change from opencv bgr default back to rgb
pimg = pimg[:, :, [2,1,0]]
vimg = vimg[:, :, [2,1,0]]
aspect = (pimg.shape[0]+vimg.shape[0])/float(np.max([pimg.shape[1], vimg.shape[1]]))

fig, ax = plt.subplots(1,2, figsize=(20,20/(2*aspect)))
vax = ax[0]
pax = ax[1]

vim = vax.imshow(vimg, origin='upper')
vpa = pointAnnotator(vim, vax)
vasculature_lanmark_points
vasculature_landmark_points = helpers.points_from_path(vimg_path, chosen_probe_image_name, 'landmark')
ppa.loadPoints(vasculature_landmark_points)

pim = pax.imshow(pimg, origin='upper')
ppa = pointAnnotator(pim, pax)
probe_landmark_points = helpers.points_from_path(pimg_path, chosen_vasculature_image_name, 'landmark')
ppa.loadPoints(probe_landmark_points)

plt.tight_layout(pad=1, w_pad=0, h_pad=0)

In [ ]:
#Compute affine transform to warp probe insertion image to vasculature image coordinates

#reshape points into [numpoints, 2] arrays
ppoints = ppa.points #np.stack((ppa.xs, ppa.ys)).astype(np.float32).T
vpoints = vpa.points #np.stack((vpa.xs, vpa.ys)).astype(np.float32).T

#get perspective transform between images and apply it to probe insertion image
perspective = cv2.findHomography(ppoints, vpoints)
pimg_transform = cv2.warpPerspective(pimg, perspective[0], (vimg.shape[1], vimg.shape[0]))

#transform probe insertion points to use in local warping or plot on transformed image
ppoints_transform = cv2.perspectiveTransform(np.array([ppoints.astype(np.float32)]), perspective[0])
ppoints_transform = np.squeeze(ppoints_transform)

#perform Delaunay triangle local warp if specified
if localWarp:
    pimg_transform = warpImage.warpImage(vimg, pimg_transform, vpoints, ppoints_transform)

#plot overlay of vasculature image and transformed image to compare
fig, ax = plt.subplots(figsize=(10,10/aspect))
ax.imshow(vimg, alpha=0.5)
ax.imshow(255-pimg_transform, alpha=0.5)


In [10]:
#Select location of probe insertion points in this warped image
fig, ax = plt.subplots(1,2, figsize=(20,20/(2*aspect)))
paffax = ax[0]
vax2 = ax[1]

paff_im = paffax.imshow(pimg_transform, origin='upper')
paffp = pointAnnotator(paff_im, paffax)
probe_insertion_points = helpers.points_from_path(pimg_path, chosen_vasculature_image_name, chosen_probe_image_name, 'insertion')
paffp.loadPoints(insertion_points)

paffpoints = paffp.points  # np.stack((paffp.xs, paffp.ys)).astype(np.float32).T

vim2 = vax2.imshow(vimg, origin='upper')
vpa = pointCopier(vim2, vax2)
vpa.loadPoints(insertion_points)

NameError: name 'aspect' is not defined

In [ ]:
print(paffpoints)#I think we need to update this before writing?
#also lets do one in the ISI pixel coords to make it easy - but we need to associate the targets with the vasculature image they were matched from

In [ ]:
# save points and transformation matrix
save_path = cfg.get_save_path(pimgFileName)
try:
    current_npz = np.load(save_path)
    save_dict = dict(current_npz)
except FileNotFoundError as E:
    save_dict = {}

save_dict['vasculatureImageKeypoints'] = vpoints
save_dict['probeImageKeypoints'] = ppoints
save_dict['vasculatureToProbeInsertionTransform'] = perspective[0]
save_dict['probeInsertionPointsTransformed'] = paffpoints
save_dict[chosen_vasculature_image_name+'_landmark_points'] = vpoints
save_dict[chosen_probe_image_name+'_landmark_points'] = ppoints
save_dict[chosen_probe_image_name+'_to_'+chosen_vasculature_image_name+'_transform'] = perspective[0]
save_dict[chosen_probe_image_name+'_to_'+chosen_vasculature_image_name+'_insertion_points'] = paffpoints

np.savez(save_path, **save_dict)



In [ ]:
# save notebook as html
saveDirectory = os.path.dirname(pimgFileName)
os.system('jupyter nbconvert probeInsertionLocator.ipynb --output ' + os.path.join(saveDirectory, 'probeInsertionNotebook.html'))